# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686560


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:05<01:13,  2.63s/it]

Rendering models:  10%|█         | 3/30 [00:05<00:51,  1.89s/it]

Rendering models:  13%|█▎        | 4/30 [00:05<00:36,  1.39s/it]

Rendering models:  17%|█▋        | 5/30 [00:05<00:25,  1.04s/it]

...

Rendering models:  90%|█████████ | 27/30 [00:10<00:00,  6.11it/s]

Rendering models:  93%|█████████▎| 28/30 [00:11<00:00,  5.93it/s]

Rendering models:  97%|█████████▋| 29/30 [00:11<00:00,  5.08it/s]

Rendering models: 100%|██████████| 30/30 [00:11<00:00,  5.39it/s]

ElisabethB                            0.002129
ellianderjoy                          0.000349
not-logged-in-7e36d1f05e1fdd4c2b64    0.000257
kjkavene                              0.000254
not-logged-in-3e831daf31432a82f496    0.012025
jmfranci                              0.000519
jfhood                                0.000321
not-logged-in-ef704c739eb8412f355e    0.000946
mab114                                0.012153
lsautter                              0.000313
wrxtminecrafter                       0.000739
coyotes54                             0.000250
nora.zoo                              0.000338
Giantjack14                           0.004193
withadnotat                           0.000299
not-logged-in-d04ed8c60cd5453cb408    0.000245
Planetme                              0.000797
acapirala                             0.000264
Susymuller                            0.000368
not-logged-in-90294de164951e098d0b    0.017691
not-logged-in-8700ff8251e0704f745c    0.000259
saferguson   

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())